In [ ]:
import mysql.connector
# I know this is bad practice, but I did not want to explicitly import every funcion I am using
# Maybe refactoring would be in order for a real world project, but I think it will suffice in this case
from data.execution import *

##### The below will ask for two inputs Database User and Password for local host respectively


In [ ]:
cnx, cursor, prep_cursor = init_database()

In [ ]:
insert_data(cnx, cursor)

##### Adding Sales Reports


**Creating and Calling the Virtual Tables**


In [ ]:
order_views, over90orders, popular_items = run_views(cursor)

In [ ]:
print("Order Views Response")
for data in order_views:
    print(
        f"orderId: {data[0]} | Order Quantity: {data[1]} | Order Total: ${data[2]}")

In [ ]:
print("Orders Over 90 Dollars")
for data in over90orders:
    print(
        f"Customer: {data[1]} | Order Total: ${data[4]} | Order Type: {data[3]}")

In [ ]:
print("Popular Menu Items")
for data in popular_items:
    print(f'Item: {data[0]}')

**Creating and Calling Procedures**


In [ ]:
run_proceedures(cursor)

In [ ]:
query = call_procedure(cursor, 'GetMaxQuantity')
print(f"The Maximum Order Size is {query[0][0][0]}")

In [ ]:
query = call_procedure(cursor, 'CancelOrder', (9, 'message'))
query

In [ ]:
response = run_prepared_statement(prep_cursor, (4, ))
for order in response:
    print(f"Customer #: {order[0]} | Quantity: {order[1]} | Total: {data[2]}")

##### Table Booking System


In [ ]:
create_booking_procedures(cursor)

In [ ]:
call_procedure(cursor, "CheckBooking")
call_procedure(cursor, "AddValidBooking")

##### Data Analysis Testing


In [2]:
import pandas as pd
import numpy as np

df = pd.read_excel("./data/lemon_dat.xlsx")

In [3]:
df.shape

(21000, 21)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row Number     21000 non-null  int64         
 1   Order ID       21000 non-null  object        
 2   Order Date     21000 non-null  datetime64[ns]
 3   Delivery Date  21000 non-null  datetime64[ns]
 4   Customer ID    21000 non-null  object        
 5   Customer Name  21000 non-null  object        
 6   City           21000 non-null  object        
 7   Country        21000 non-null  object        
 8   Postal Code    21000 non-null  object        
 9   Country Code   20958 non-null  object        
 10   Cost          21000 non-null  float64       
 11  Sales          21000 non-null  float64       
 12  Quantity       21000 non-null  int64         
 13  Discount       21000 non-null  float64       
 14  Delivery Cost  21000 non-null  float64       
 15  Course Name    2100

In [5]:
df.rename(columns={" Cost": "Cost"}, inplace=True)
df.head(3)

,Row Number,Order ID,Order Date,Delivery Date,Customer ID,Customer Name,City,Country,Postal Code,Country Code,...,Sales,Quantity,Discount,Delivery Cost,Course Name,Cuisine Name,Starter Name,Desert Name,Drink,Sides
0,1,54-366-6861,2020-06-15,2020-03-26,72-055-7985,Laney Fadden,Daruoyan,China,993-0031,CN,...,187.5,2,20.00,60.51,Greek salad,Greek,Olives,Greek yoghurt,Athens White wine,Tapas
1,2,63-761-3686,2020-08-25,2020-07-17,65-353-0657,Giacopo Bramich,Ongjin,North Korea,216282,KP,...,352.5,1,15.00,96.75,Bean soup,Italian,Flatbread,Ice cream,Corfu Red Wine,Potato salad
2,3,65-351-6434,2021-08-17,2020-04-24,90-876-6799,Lia Bonar,Quince Mil,Peru,663246,PE,...,112.5,3,10.52,36.37,Pizza,Italian,Minestrone,Cheesecake,Italian Coffee,Bruschetta


##### Feature Engineering


In [6]:
df[["First Name", "Last Name"]] = df["Customer Name"].str.split(
    " ", expand=True, n=1)

In [7]:
df["Profit"] = df["Sales"] - df["Cost"]

In [10]:
df[["Quantity", "Profit"]]

,Quantity,Profit
0,2,62.500
1,1,117.500
2,3,37.500
3,3,110.000
4,2,160.000
...,...,...
20995,3,62.310
20996,2,72.045
20997,1,50.800
20998,2,46.565


##### Building the Visualizations

**Using Pandas for Data Cleaning**

**Using Plotly for Data Visualizations**


In [103]:
import plotly.graph_objects as go
import plotly.express as px
config = {
    'modeBarButtonsToRemove': ['zoom', 'zoomIn', 'zoomOut', 'select', 'lasso', "autoScale2d"],
}

**Task 1**


In [104]:
# Data Cleaning
cust_sales = df.copy()
cust_sales = cust_sales.loc[cust_sales["Sales"] >= 70]
cust_sales = cust_sales.groupby("Customer Name", as_index=False).sum(
    numeric_only=True)[["Customer Name", "Sales"]].sort_values("Sales", ascending=False)
cust_sales.head(2)

,Customer Name,Sales
913,Tasia Fautly,5007.3
740,Noni Quickenden,4992.3


In [105]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=cust_sales["Customer Name"],
    y=cust_sales["Sales"],
    hovertemplate="""
    <b>Customer:</b> %{x}<br>
    <b>Total Sales:</b> %{y:$,.2f}
    <extra></extra>
    """,
    texttemplate="%{y:$,.2f}"
))


fig.update_xaxes(range=[-1, 10])
fig.update_yaxes(fixedrange=True)

fig.update_layout(
    dragmode="pan",
    title='Customer Sales',
    xaxis_title='Customer',
    yaxis_title='Total Sales ($ USD)'
)

fig.show(config=config)

**Task 2**


In [106]:
# Filtering the data
profit_df = df.copy()
profit_df = profit_df.groupby("Order Date", as_index=False).sum(
    numeric_only=True).sort_values("Order Date")
profit_df = profit_df.loc[profit_df["Order Date"] < "2023-01-01"]

In [107]:
profit_df

,Order Date,Row Number,Cost,Sales,Quantity,Discount,Delivery Cost,Profit
0,2019-06-14,224973,3202.58,4803.870,42,1036.48,1988.07,1601.290
1,2019-06-23,230475,3152.58,4728.870,42,823.65,1416.87,1576.290
2,2019-06-26,680253,9351.52,14027.280,189,3075.23,1567.02,4675.760
3,2019-06-27,448833,6315.71,9473.565,105,2044.03,1078.14,3157.855
4,2019-07-03,432117,6256.15,9384.225,105,2153.12,1438.29,3128.075
...,...,...,...,...,...,...,...,...
692,2022-12-24,229782,3066.88,4600.320,21,1092.42,926.52,1533.440
693,2022-12-25,230223,3109.31,4663.965,63,1102.62,179.76,1554.655
694,2022-12-28,218022,2956.50,4434.750,21,1124.66,267.12,1478.250
695,2022-12-29,223881,3185.74,4778.610,42,1124.66,572.04,1592.870


In [108]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=profit_df["Order Date"],
    y=profit_df["Profit"],
    mode="lines",
    hovertemplate="""
    <b>Profit:</b> %{y:$,.2f}<br>
    <b>Date:</b> %{x}
    <extra></extra>
    """
))

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector_buttons=list([
        dict(count=1, label="1m", step="month", stepmode="backward"),
        dict(count=3, label="3m", step="month", stepmode="backward"),
        dict(count=6, label="6m", step="month", stepmode="backward"),
        dict(count=1, label="1y", step="year", stepmode="backward"),
        dict(count=2, label="2yr", step="year", stepmode="backward"),
        dict(count=3, label="3yr", step="year", stepmode="backward"),
    ]),
)

fig.update_layout(
    dragmode="pan",
    title='Profit Chart',
    xaxis_title='Order Date',
    yaxis_title='Total Profit ($ USD)',
    yaxis_tickformat='$,.2f'
)

fig.show(config=config)

**Task 3**


In [109]:
bubble_df = df.copy()
bubble_df = bubble_df.groupby(
    "Customer Name", as_index=False).sum(numeric_only=True)

In [110]:
bubble_df

,Customer Name,Row Number,Cost,Sales,Quantity,Discount,Delivery Cost,Profit
0,Aaron Cromley,227640,3107.51,4661.265,63,1183.24,1396.08,1553.755
1,Abigail Struis,213906,2971.50,4457.250,21,1036.48,444.36,1485.750
2,Adah Adamczewski,212415,3208.20,4812.300,63,884.79,1950.27,1604.100
3,Adaline Reichartz,227724,2956.50,4434.750,63,937.07,1209.81,1478.250
4,Adamo Rumens,217497,3208.20,4812.300,42,969.88,556.08,1604.100
...,...,...,...,...,...,...,...,...
995,Zita Coldbath,215859,3087.58,4631.370,42,1036.48,1469.58,1543.790
996,Zolly Heinzel,217770,3066.88,4600.320,21,905.30,506.10,1533.440
997,Zora Hanalan,212457,3152.58,4728.870,42,1014.77,514.92,1576.290
998,Zorana Diggin,215124,3058.33,4587.495,63,1175.08,1770.72,1529.165


In [111]:
fig = px.scatter(
    bubble_df,
    x="Profit",
    y="Sales",
    color="Customer Name",
    size="Quantity",
    hover_data={
        "Customer Name": False,
        "Profit": ":$,.2f",
        "Sales": ":$,.2f",
    },
    hover_name="Customer Name"
)

fig.update_layout(
    showlegend=False,
    title='Sales Buble Chart (w/ size=Quantity)',
    yaxis_title='Total Sales ($ USD)',
    yaxis_tickformat="$,.2f",
    xaxis_title='Total Profit ($ USD)',
    xaxis_tickformat="$,.2f"
)

fig.show(config={
    'modeBarButtonsToRemove': ['zoomIn', 'zoomOut', 'select', 'lasso', "autoScale2d"],
})

**Task 4**


In [124]:
cuisine_df = df.copy()
cuisine_df["Year"] = cuisine_df["Order Date"].dt.year
cuisine_df = cuisine_df.loc[cuisine_df["Year"].between(
    2020, 2022, inclusive="both")]
cuisine_df = cuisine_df.groupby(["Cuisine Name", "Year"], as_index=False).sum(
    numeric_only=True).sort_values(["Year", "Sales"], ascending=False)

In [125]:
fig = px.histogram(
    cuisine_df,
    x="Year",
    y="Profit",
    color="Cuisine Name",
    barmode='group',
    hover_data={
        "Cuisine Name": False,
        "Year": True,
        "Profit": ":$,.2f"
    },
    hover_name="Cuisine Name"
)

fig.update_traces(texttemplate="%{y:$,.2f}")

fig.update_layout(
    title='Yearly Profit by Cuisine',
    yaxis_title='Total Profit ($ USD)',
    yaxis_tickformat="$,.2f",
    xaxis_title='Year'
)

fig.show(config={
    'modeBarButtonsToRemove': ['zoomIn', 'zoomOut', 'select', 'lasso', "autoScale2d"],
})

**Task 5**


In [143]:
df.columns

Index(['Row Number', 'Order ID', 'Order Date', 'Delivery Date', 'Customer ID',
       'Customer Name', 'City', 'Country', 'Postal Code', 'Country Code',
       'Cost', 'Sales', 'Quantity', 'Discount', 'Delivery Cost', 'Course Name',
       'Cuisine Name', 'Starter Name', 'Desert Name', 'Drink', 'Sides',
       'First Name', 'Last Name', 'Profit'],
      dtype='object')

In [144]:
test = df.groupby(["Customer Name", "Country", "City", "Order Date",
                  "Order ID"], as_index=False).sum(numeric_only=True)
test[test["Customer Name"] == "Doug Logg"]

,Customer Name,Country,City,Order Date,Order ID,Row Number,Cost,Sales,Quantity,Discount,Delivery Cost,Profit
271,Doug Logg,Russia,Mozhaysk,2022-05-27,40-599-5065,227157,3225.5,4838.25,42,1196.72,1199.1,1612.75
